In [1]:
import rclpy as rp
from turtlesim.msg import Pose

In [10]:
rp.init()
test_node = rp.create_node('sub_test')

RuntimeError: Context.init() must only be called once

In [4]:
def callback(data):
    print("--->")
    print("/turtle1/pose : ", data)
    print("X : ", data.x)
    print("Y : ", data.y)
    print("Theta : ", data.theta)

In [9]:
test_node.create_subscription(Pose, "/turtle1/pose", callback, 10)

In [7]:
rp.spin_once(test_node)

--->
/turtle1/pose :  turtlesim.msg.Pose(x=5.544444561004639, y=5.544444561004639, theta=0.0, linear_velocity=0.0, angular_velocity=0.0)
X :  5.544444561004639
Y :  5.544444561004639
Theta :  0.0


In [16]:
cnt = 0 
def callback(data):
    global cnt
    cnt += 1
    print('>', cnt, " -> X : ", data.x, ", Y : ", data.y)
    if cnt>3:
        raise Exception("Subscription stop")

In [13]:
test_node.create_subscription(Pose, "/turtle1/pose", callback, 10)

In [17]:
rp.spin(test_node)

--->
/turtle1/pose :  turtlesim.msg.Pose(x=2.1847078800201416, y=3.9078450202941895, theta=1.4079999923706055, linear_velocity=0.0, angular_velocity=0.0)
X :  2.1847078800201416
Y :  3.9078450202941895
Theta :  1.4079999923706055
--->
/turtle1/pose :  turtlesim.msg.Pose(x=2.1847078800201416, y=3.9078450202941895, theta=1.4079999923706055, linear_velocity=0.0, angular_velocity=0.0)
X :  2.1847078800201416
Y :  3.9078450202941895
Theta :  1.4079999923706055
--->
/turtle1/pose :  turtlesim.msg.Pose(x=2.1847078800201416, y=3.9078450202941895, theta=1.4079999923706055, linear_velocity=0.0, angular_velocity=0.0)
X :  2.1847078800201416
Y :  3.9078450202941895
Theta :  1.4079999923706055
--->
/turtle1/pose :  turtlesim.msg.Pose(x=2.1847078800201416, y=3.9078450202941895, theta=1.4079999923706055, linear_velocity=0.0, angular_velocity=0.0)
X :  2.1847078800201416
Y :  3.9078450202941895
Theta :  1.4079999923706055
> 1  -> X :  2.1847078800201416 , Y :  3.9078450202941895
--->
/turtle1/pose :  

Exception: Subscription stop

In [19]:
import rclpy as rp
from geometry_msgs.msg import Twist

rp.init()
test_node = rp.create_node("pub_test")

RuntimeError: Context.init() must only be called once

In [20]:
msg = Twist()
print(msg)

geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0))


In [29]:
msg.linear.x = 2.0
print(msg)

geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=2.0, y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0))


In [30]:
pub = test_node.create_publisher(Twist, "/turtle1/cmd_vel", 10)
pub.publish(msg)

InvalidHandle: cannot use Destroyable because destruction was requested

In [26]:
cnt = 0

def timer_callback():
    global cnt

    cnt += 1

    print(cnt)
    pub.publish(msg)

    if cnt > 3:
        raise Exception("Publisher Stop")

In [27]:
timer_period = 0.1
timer = test_node.create_timer(timer_period, timer_callback)
rp.spin(test_node)

1


InvalidHandle: cannot use Destroyable because destruction was requested

In [28]:
test_node.destroy_node()

In [33]:
import rclpy as rp
from rclpy.node import Node
from geometry_msgs.msg import Twist
from turtlesim.msg import Pose

class Demo(Node):
    def __init__(self):
        super().__init__('sub_test')
        # 엔티티들은 노드의 속성으로 보관해야 GC에 안 사라집니다.
        self.pub = self.create_publisher(Twist, '/turtle1/cmd_vel', 10)
        self.sub = self.create_subscription(Pose, '/turtle1/pose', self.cb_pose, 10)
        self.cnt = 0
        self.timer = self.create_timer(0.1, self.cb_timer)  # 10Hz

    def cb_pose(self, msg: Pose):
        # 필요시: self.get_logger().info(f"x={msg.x:.2f} ...")
        pass

    def cb_timer(self):
        self.cnt += 1
        msg = Twist()
        msg.linear.x = 2.0
        msg.angular.z = 1.8
        self.pub.publish(msg)

        if self.cnt >= 100:           # 3초 후 정지
            self.timer.cancel()       # 예외 던지지 말고 타이머 종료
            msg = Twist()             # 0 속도로 멈춤
            self.pub.publish(msg)
            self.get_logger().info("timer stopped")

# --- 실행 ---
# (노트북에서 이전 init이 있었다면 정리)
try: rp.shutdown()
except Exception: pass

rp.init()
node = Demo()
try:
    rp.spin(node)     # 블로킹. 백그라운드로 돌리고 싶으면 Executor+thread 사용
finally:
    node.destroy_node()
    rp.shutdown()


[INFO] [1758173504.064694302] [sub_test]: timer stopped


KeyboardInterrupt: 

In [34]:
import rclpy as rp
from turtlesim.srv import TeleportAbsolute

rp.init()
test_node = rp.create_node("client_test")

In [54]:
service_name = "/turtle1/teleport_absolute"
cli = test_node.create_client(TeleportAbsolute, service_name)

In [55]:
req = TeleportAbsolute.Request()
req

turtlesim.srv.TeleportAbsolute_Request(x=0.0, y=0.0, theta=0.0)

In [56]:
req.x = 1.
req.y = 1.
req.theta = 3.14

req

turtlesim.srv.TeleportAbsolute_Request(x=1.0, y=1.0, theta=3.14)

In [58]:
req.x = 3.

cli.call_async(req)
rp.spin_once(test_node)

In [59]:
req.y = float(9)

while not cli.wait_for_service(timeout_sec=1.0):
    print("Waiting for service")

cli.call_async(req)
rp.spin_once(test_node)